In [302]:
%load_ext autoreload
%autoreload 2
from bytedmerlin.devbox.worker import launch_worker
import pyscf
import time
import numpy as np
from pyscf import lib, scf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [364]:
atom = '''
 C -0.9825    0.0000    0.0001
 C -0.2850    1.2079    0.0001
 C -0.2852   -1.2079    0.0000
 C -2.4741    0.0001   -0.0002
 C  1.1099    1.2080   -0.0001
 C  1.1097   -1.2080    0.0000
 C  1.8072    0.0000   -0.0001
 H -0.8169    2.1557   -0.0017
 H -0.8172   -2.1556   -0.0018
 H -2.8694   -0.8801    0.5183
 H -2.8694    0.8808    0.5175
 H -2.8495   -0.0004   -1.0283
 H  1.6531    2.1484   -0.0007
 H  1.6529   -2.1485   -0.0005
 H  2.8933   -0.0001   -0.0004
'''

bas='def2tzvpp'
mol = pyscf.M(atom=atom, basis=bas, charge=0, spin=0, max_memory=32000)
# mol.output = '/dev/null'
mol.verbose = 6
mol.build()

In [375]:
@launch_worker(cpu=6, memory=30, gpu=1)
def test(xc):
    from gpu4pyscf.dft import rks,uks
    from gpu4pyscf import properties
    
    mf = rks.RKS(mol, xc=xc).density_fit(auxbasis='def2-universal-jkfit')
    mf.grids.level=6
    mf.conv_tol = 1e-12
    e_dft = mf.kernel()

    # h = mf.Hessian()
    # # h.auxbasis_response = 2
    # h.kernel()
    
    # polarizability = properties.polarizability.eval_polarizability(mf)
    nmr_shielding = properties.shielding.eval_shielding(mf)
    # freq, intensity = properties.ir.eval_ir_freq_intensity(mf, h)
    
    # print(polarizability)
    print(nmr_shielding)
    nmr_total = nmr_shielding[0]+nmr_shielding[1]
    for iatm in range(mol.natm):
        print(mol.atom_symbol(iatm), nmr_total[iatm].trace()/3)
    # print(nmr_total[0].trace()/3, nmr_total[1].trace()/3, nmr_total[2].trace()/3)
    # print(freq, intensity)


In [376]:
test("b3lyp")

current version: 1.0.0.410
latest  version: 1.0.0.409
检测到mlx cli非最新版本，自动更新


time="2024-03-12T14:31:06+08:00" level=info msg="Worker resources" CPU=6 GPU=1 Memory=30 Node=1 Type=Tesla-V100-GENERAL-32GB
time="2024-03-12T14:31:12+08:00" level=info msg="Worker 0 has been scheduled and is starting"
time="2024-03-12T14:31:15+08:00" level=info msg="Worker 0 running" host="fdbd:dc02:16:717::19" pod="fdbd:dc02:16:717:1300::6a"


/usr/local/lib/python3.9/dist-packages/gpu4pyscf/lib/cutensor.py:138: UserWarning: using cupy as the tensor contraction engine.
  warnings.warn(f'using {contract_engine} as the tensor contraction engine.')


******** <class 'gpu4pyscf.df.df_jk.DFRKS'> ********
method = DFRKS
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'gpu4pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = False
chkfile to save SCF result = /tmp/tmp_dj9gz8c
max_memory 32000 MB (current use 1040 MB)
XC library pyscf.dft.libxc version 6.1.0
    unable to decode the reference due to https://github.com/NVIDIA/cuda-python/issues/29
XC functionals = b3lyp
/usr/local/lib/python3.9/dist-packages/pyscf/dft/libxc.py:1102: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the 

time="2024-03-12T14:34:41+08:00" level=info msg="Finished execution"
